In [1]:
!pip install mysql-connector-python
!pip install requests
!pip install --upgrade mysql-connector-python
!pip install cryptography


[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import requests
import pyspark.sql.functions as funct
from pyspark.sql.types import *
import mysql.connector as msql
from mysql.connector import Error
import pandas as pd
import cryptography
import credentials as cred



# **Data Extraction from the Loan Data _API_**

###  Extracting the data from the API endpoint and storing it in a Spark dataframe.  The API url is provided in the Capstone Guidance Document.

In [3]:
import requests


url = 'https://raw.githubusercontent.com/platformps/LoanDataset/main/loan_data.json'



response = requests.get(url)
response.headers["Content-Type"]
api_results = response.json()
print(f"Status code: {response.status_code}")
print(f"{response.headers}")

Status code: 200
{'Connection': 'keep-alive', 'Content-Length': '3943', 'Cache-Control': 'max-age=300', 'Content-Security-Policy': "default-src 'none'; style-src 'unsafe-inline'; sandbox", 'Content-Type': 'text/plain; charset=utf-8', 'ETag': 'W/"a8013af2b71077e4636d6d2d52ae3431aa0fae0b32dc61b867578cf1100b198b"', 'Strict-Transport-Security': 'max-age=31536000', 'X-Content-Type-Options': 'nosniff', 'X-Frame-Options': 'deny', 'X-XSS-Protection': '1; mode=block', 'X-GitHub-Request-Id': 'E388:1492DA:3CB737:414A39:668C9E91', 'Content-Encoding': 'gzip', 'Accept-Ranges': 'bytes', 'Date': 'Tue, 09 Jul 2024 02:24:12 GMT', 'Via': '1.1 varnish', 'X-Served-By': 'cache-chi-kigq8000037-CHI', 'X-Cache': 'HIT', 'X-Cache-Hits': '0', 'X-Timer': 'S1720491852.021088,VS0,VE1', 'Vary': 'Authorization,Accept-Encoding,Origin', 'Access-Control-Allow-Origin': '*', 'Cross-Origin-Resource-Policy': 'cross-origin', 'X-Fastly-Request-ID': '8fdad8a735cdd3db04acbec3f887c2d0b31917aa', 'Expires': 'Tue, 09 Jul 2024 02:29:

### Creating a spark dataframe, loan_application_df, from the API response.

In [4]:
# Create Spark DataFrame
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("LoanApplication").getOrCreate()

# Create DataFrame from API results
loan_application_df = spark.createDataFrame(api_results)

# Show the first few rows of the DataFrame
loan_application_df.show(5)


+--------------+------------------+--------------+----------+------------+------+------+-------+-------------+-------------+
|Application_ID|Application_Status|Credit_History|Dependents|   Education|Gender|Income|Married|Property_Area|Self_Employed|
+--------------+------------------+--------------+----------+------------+------+------+-------+-------------+-------------+
|      LP001002|                 Y|             1|         0|    Graduate|  Male|medium|     No|        Urban|           No|
|      LP001003|                 N|             1|         1|    Graduate|  Male|medium|    Yes|        Rural|           No|
|      LP001005|                 Y|             1|         0|    Graduate|  Male|   low|    Yes|        Urban|          Yes|
|      LP001006|                 Y|             1|         0|Not Graduate|  Male|   low|    Yes|        Urban|           No|
|      LP001008|                 Y|             1|         0|    Graduate|  Male|medium|     No|        Urban|           No|


In [5]:
#Reviewing the schema along with the Data that has been loaded
loan_application_df.printSchema()
loan_application_df.show(15)

root
 |-- Application_ID: string (nullable = true)
 |-- Application_Status: string (nullable = true)
 |-- Credit_History: long (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- Education: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Income: string (nullable = true)
 |-- Married: string (nullable = true)
 |-- Property_Area: string (nullable = true)
 |-- Self_Employed: string (nullable = true)

+--------------+------------------+--------------+----------+------------+------+------+-------+-------------+-------------+
|Application_ID|Application_Status|Credit_History|Dependents|   Education|Gender|Income|Married|Property_Area|Self_Employed|
+--------------+------------------+--------------+----------+------------+------+------+-------+-------------+-------------+
|      LP001002|                 Y|             1|         0|    Graduate|  Male|medium|     No|        Urban|           No|
|      LP001003|                 N|             1|         1|    G

### Use the schema described above to create a Spark dataframe that will maintain the structure types, structure fields, string and long types

In [6]:
# Create Spark DataFrame with the specified schema
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, LongType

# Define the schema
schema = StructType([
    StructField("Application_ID", StringType(), True),
    StructField("Application_Status", StringType(), True),
    StructField("Credit_History", LongType(), True),
    StructField("Dependents", StringType(), True),
    StructField("Education", StringType(), True),
    StructField("Gender", StringType(), True),
    StructField("Income", StringType(), True),
    StructField("Married", StringType(), True),
    StructField("Property_Area", StringType(), True),
    StructField("Self_Employed", StringType(), True)
])


# Create DataFrame from API results with the specified schema
loan_application_df = spark.createDataFrame(api_results, schema)

# Show the first few rows of the DataFrame
loan_application_df.show(5)
loan_application_df.describe().show()


+--------------+------------------+--------------+----------+------------+------+------+-------+-------------+-------------+
|Application_ID|Application_Status|Credit_History|Dependents|   Education|Gender|Income|Married|Property_Area|Self_Employed|
+--------------+------------------+--------------+----------+------------+------+------+-------+-------------+-------------+
|      LP001002|                 Y|             1|         0|    Graduate|  Male|medium|     No|        Urban|           No|
|      LP001003|                 N|             1|         1|    Graduate|  Male|medium|    Yes|        Rural|           No|
|      LP001005|                 Y|             1|         0|    Graduate|  Male|   low|    Yes|        Urban|          Yes|
|      LP001006|                 Y|             1|         0|Not Graduate|  Male|   low|    Yes|        Urban|           No|
|      LP001008|                 Y|             1|         0|    Graduate|  Male|medium|     No|        Urban|           No|


# Establishing DB connection to create the CDW_SAPP_loan_application table as per the guidance

In [8]:
# Prepare for MySQL connection
host = cred.host
user = cred.user
password = cred.password
database = "creditcard_capstone"

# Connect to MySQL and create table
try:
    conn = msql.connect(host=host, user=user, password=password, database=database)
    if conn.is_connected():
        cursor = conn.cursor()
        
        # Create table with the specified schema
        create_table_query = """
        CREATE TABLE IF NOT EXISTS CDW_SAPP_loan_application (
            Application_ID VARCHAR(255),
            Application_Status VARCHAR(255),
            Credit_History BIGINT,
            Dependents VARCHAR(255),
            Education VARCHAR(255),
            Gender VARCHAR(255),
            Income VARCHAR(255),
            Married VARCHAR(255),
            Property_Area VARCHAR(255),
            Self_Employed VARCHAR(255)
        )
        """
        cursor.execute(create_table_query)
        print("Table created successfully")

except Error as e:
    print(f"Error: {e}")
    print(f"Error type: {type(e).__name__}")
    print(f"Error args: {e.args}")

finally:
    if 'conn' in locals() and conn.is_connected():
        cursor.close()
        conn.close()
        print("MySQL connection is closed")
        
        

Table created successfully
MySQL connection is closed


#### This next step will write the spark dataframe, loan_application_df to the MYSQL creditcard_capstone.CDW_SAPP_loan_application

In [9]:
loan_application_df.write.format("jdbc") \
  .mode("append") \
  .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
  .option("dbtable", "creditcard_capstone.CDW_SAPP_loan_application") \
  .option("user", cred.user) \
  .option("password", cred.password) \
  .save()


### Converting quickly to pandas_df to see appearance of data and appearance of table columns

In [10]:
pandas_df = loan_application_df.toPandas()
pandas_df.head()

,Application_ID,Application_Status,Credit_History,Dependents,Education,Gender,Income,Married,Property_Area,Self_Employed
0,LP001002,Y,1,0,Graduate,Male,medium,No,Urban,No
1,LP001003,N,1,1,Graduate,Male,medium,Yes,Rural,No
2,LP001005,Y,1,0,Graduate,Male,low,Yes,Urban,Yes
3,LP001006,Y,1,0,Not Graduate,Male,low,Yes,Urban,No
4,LP001008,Y,1,0,Graduate,Male,medium,No,Urban,No


In [1]:
!pip list


Package                   Version
------------------------- -----------
asttokens                 2.4.1
attrs                     23.2.0
blinker                   1.8.2
certifi                   2024.2.2
cffi                      1.16.0
charset-normalizer        3.3.2
click                     8.1.7
colorama                  0.4.6
comm                      0.2.2
contourpy                 1.2.1
cryptography              42.0.8
cycler                    0.12.1
dash                      2.17.0
dash-core-components      2.0.0
dash-html-components      2.0.0
dash-table                5.0.0
debugpy                   1.8.1
decorator                 5.1.1
executing                 2.0.1
fastjsonschema            2.19.1
findspark                 2.0.1
Flask                     3.0.3
fonttools                 4.51.0
grpcio                    1.64.1
grpcio-tools              1.64.1
idna                      3.7
importlib_metadata        7.1.0
ipykernel                 6.29.4
ipython              